In [ ]:
# @title Installing Packages
!pip install pdfplumber sentence-transformers
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install pymupdf langchain faiss-cpu sentence-transformers tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 63.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.5 MB/s eta 0:00:00


In [ ]:
pip install pymupdf langchain faiss-cpu sentence-transformers tiktoken

In [ ]:
# @title import nltk
# nltk.download('punkt')
import nltk
nltk.download('punkt', download_dir='/usr/local/nltk_data')

# Make sure the environment can find it
nltk.data.path.append('/usr/local/nltk_data')


[nltk_data] Downloading package punkt to /usr/local/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# @title FIle Doc Add
folder_path = '/content/pdf_folder'
pdf_folder = '/content/pdf_folder'

In [ ]:
mkdir pdf_folder

In [ ]:
# @title Layout-Aware PDF → Markdown

import fitz  # PyMuPDF
import re
from pathlib import Path

def pdf_to_markdown_layout_aware(pdf_path: str, output_md: str = None):
    """
    Convert PDF to structured Markdown using layout info.
    Detect headings based on font size.
    Preserve structure.
    """

    doc = fitz.open(pdf_path)
    markdown_lines = []

    font_sizes = []

    # First pass: collect font sizes to estimate heading levels
    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        font_sizes.append(span["size"])

    # Determine size thresholds
    unique_sizes = sorted(set(font_sizes), reverse=True)
    if len(unique_sizes) < 3:
        h1_size = unique_sizes[0]
        h2_size = unique_sizes[1] if len(unique_sizes) > 1 else unique_sizes[0]
        h3_size = unique_sizes[-1]
    else:
        h1_size = unique_sizes[0]
        h2_size = unique_sizes[1]
        h3_size = unique_sizes[2]

    # Second pass: build markdown
    for page_number, page in enumerate(doc, start=1):
        markdown_lines.append(f"\n<!-- Page {page_number} -->\n")

        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if "lines" not in block:
                continue

            block_text = ""
            max_font_size = 0

            for line in block["lines"]:
                for span in line["spans"]:
                    text = span["text"].strip()
                    if not text:
                        continue
                    block_text += text + " "
                    max_font_size = max(max_font_size, span["size"])

            block_text = block_text.strip()
            if not block_text:
                continue

            # Heading detection
            if max_font_size >= h1_size:
                markdown_lines.append(f"# {block_text}\n")
            elif max_font_size >= h2_size:
                markdown_lines.append(f"## {block_text}\n")
            elif max_font_size >= h3_size:
                markdown_lines.append(f"### {block_text}\n")
            else:
                markdown_lines.append(f"{block_text}\n")

    markdown_text = "\n".join(markdown_lines)

    # Detect algorithm blocks
    markdown_text = detect_algorithm_blocks(markdown_text)

    if output_md:
        Path(output_md).write_text(markdown_text, encoding="utf-8")

    return markdown_text

In [ ]:
# @title Algorithm Block Detection

def detect_algorithm_blocks(md_text: str):
    """
    Wrap algorithm pseudocode blocks in Markdown code fences.
    """

    pattern = r"(algorithm .*?end [A-Za-z]+)"

    matches = re.findall(pattern, md_text, re.DOTALL)

    for match in matches:
        fenced = f"\n```pseudo\n{match.strip()}\n```\n"
        md_text = md_text.replace(match, fenced)

    return md_text

In [ ]:
!pip install -U langchain-text-splitters

In [ ]:
# @title Smart Chunking (Algorithm-Aware)

# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

def smart_markdown_chunker(md_text):

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        separators=["\n```", "\n## ", "\n# ", "\n\n", "\n", " "]
    )

    chunks = splitter.split_text(md_text)

    return chunks

In [ ]:
# @title Run the program Layout,Algorithm,Chunks

# pdf_path = "/mnt/data/DSAdata.pdf"
pdf_path = "/content/DSAdata.pdf"

# 1️ Convert to Markdown
markdown_text = pdf_to_markdown_layout_aware(pdf_path, "dsa_output.md")

# 2️ Chunk
chunks = smart_markdown_chunker(markdown_text)

In [ ]:
# @title File text into Chunks part using nlp spaCy English model

import spacy
import pdfplumber
import os

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

def extract_chunks_from_pdfs_spacy(folder_path, chunk_size=300):
    all_chunks = []
    file_sources = []

    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            with pdfplumber.open(file_path) as pdf:
                text = ""
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + " "

            doc = nlp(text)
            sentences = [sent.text.strip() for sent in doc.sents]

            chunk = ""
            for sent in sentences:
                if len(chunk) + len(sent) < chunk_size:
                    chunk += " " + sent
                else:
                    all_chunks.append(chunk.strip())
                    file_sources.append(file)
                    chunk = sent
            if chunk:
                all_chunks.append(chunk.strip())
                file_sources.append(file)

    return all_chunks, file_sources

# Run it
chunks, sources = extract_chunks_from_pdfs_spacy(pdf_folder)
print(f"Extracted {len(chunks)} chunks.")


Extracted 611 chunks.


In [ ]:
# @title Load Sentence Transformer Model and Turn text into Embeddings

from sentence_transformers import SentenceTransformer

# Load the embedding model
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

model = SentenceTransformer("BAAI/bge-small-en-v1.5")


# Embed the chunks
embeddings = model.encode(chunks, show_progress_bar=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# @title View first 10 chunks with their source
for i in range(50):
    # print(f"🔹 Chunk {i+1} (from {sources[i]}):")
    print(f"🔹 Chunk {i+1}")
    print(chunks[i])
    print("-" * 80)


🔹 Chunk 1
<!-- Page 1 -->

# DSA

### D a t a  S t r u c t u r e s  a n d  A l g o r i t h m s

Annotated Reference with Examples

Granville Barne ! Luca Del Tongo


<!-- Page 2 -->

Data Structures and Algorithms: Annotated Reference with Examples

First Edition

Copyright c ⃝ Granville Barnett, and Luca Del Tongo 2008.


<!-- Page 3 -->

This book is made exclusively available from DotNetSlackers ( http://dotnetslackers.com/ ) the place for .NET articles, and news from some of the leading minds in the software industry.


<!-- Page 4 -->
--------------------------------------------------------------------------------
🔹 Chunk 2
## Contents
--------------------------------------------------------------------------------
🔹 Chunk 3
1 Introduction 1 1.1 What this book is, and what it isn’t . . . . . . . . . . . . . . . . 1 1.2 Assumed knowledge . . . . . . . . . . . . . . . . . . . . . . . . . 1 1.2.1 Big Oh notation . . . . . . . . . . . . . . . . . . . . . . . 1 1.2.2 Imperative program

In [ ]:
print(chunks[11])

II Algorithms 62

8 Sorting 63 8.1 Bubble Sort . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 63 8.2 Merge Sort . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 63 8.3 Quick Sort . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 65 8.4 Insertion Sort . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 67 8.5 Shell Sort . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 68 8.6 Radix Sort . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 68 8.7 Summary . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 70


In [ ]:
# @title Download Chunks
# Make sure you have `chunks` already loaded (e.g., from extract_chunks_from_pdfs)
# Optional: also use `sources` if you want to track which file the chunk came from

output_file = "/content/DSAdata_chunks.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for i, chunk in enumerate(chunks):
        f.write(f"--- Chunk {i+1} ---\n")
        f.write(chunk.strip())
        f.write("\n\n")  # Add spacing between chunks

print(f"Saved {len(chunks)} chunks to {output_file}")


Saved 295 chunks to /content/DSAdata_chunks.txt


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 28.4 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np
import pickle

# Convert embeddings to float32 (FAISS requires this)
embeddings = np.array(embeddings).astype("float32")

# Get embedding dimension
dimension = embeddings.shape[1]

# # Create FAISS index (L2 distance)
# index = faiss.IndexFlatL2(dimension)

# Normalize embeddings
faiss.normalize_L2(embeddings)

index = faiss.IndexFlatIP(dimension)

# Add embeddings to index
index.add(embeddings)

print(f"Total vectors in index: {index.ntotal}")

# Save FAISS index
faiss.write_index(index, "/content/dsa_index.faiss")

print("FAISS index saved!")

Total vectors in index: 295
FAISS index saved!


In [ ]:
metadata = {
    "chunks": chunks,
}

with open("/content/dsa_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("Metadata saved as PKL!")

Metadata saved as PKL!


In [ ]:
from google.colab import files

# files.download("/content/dsa_index.faiss")
# files.download("/content/dsa_metadata.pkl")
# files.download("/content/dsa_output.md")
# files.download("/content/DSAdata_chunks.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title Load Index and Metadata and Testing

import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer

# # Load model (must match embedding model used before!)
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

# Load FAISS index
index = faiss.read_index("dsa_index.faiss")

# Load metadata
with open("dsa_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

chunks = metadata["chunks"]
# sources = metadata["sources"]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
query="What is linked list in dsa"

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 22.0 MB/s eta 0:00:00


In [ ]:
import faiss
import pickle

In [ ]:
# Load FAISS index
index = faiss.read_index("dsa_index.faiss")

# Load metadata
with open("dsa_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

chunks = metadata["chunks"]

In [ ]:
import numpy as np

In [ ]:
def search1(query, top_k=5):
    # query_vector = model.encode([query]).astype("float32")
    query_vector = np.array([embeddings]).astype("float32")

    distances, indices = index.search(query_vector, top_k)

    results = []
    for i in indices[0]:
        results.append({
            "chunk": chunks[i],
            # "source": sources[i]
        })

    return results

In [ ]:
results = search1(query)
results

[{'chunk': 'As such, linked lists in DSA have the following characteristics:\n1. Insertion is O(1)\n2. Deletion is O(n)\n3. Searching is O(n)\nOut of the three operations the one that stands out is that of insertion.'},
 {'chunk': 'When\nyou use an API like that of DSA and you see a general purpose method that\naddsanodetothelist,youcanassumethatyouareaddingthenodetothetail\nof the list not the head. Adding a node to a singly linked list has only two cases:\n1.'},
 {'chunk': 'In DSA our implementations of linked lists always maintain head and tail\npointers so that insertion at either the head or tail of the list is a constant\ntime operation. Random insertion is excluded from this and will be a linear\noperation.'},
 {'chunk': 'Singly linked lists should be used when you are only performing basic in-\nsertions. In general doubly linked lists are more accommodating for non-trivial\noperations on a linked list. We recommend the use of a doubly linked list when you require forwards\nand 

In [ ]:
import requests

url = "URL"

payload = {
    "text": f"""{query}"""
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

embed=response.json()

print("Status Code:", response.status_code)
print("Response JSON:")
print(response.json())

Status Code: 200
Response JSON:
{'data': [[-0.0908203125, -0.01268768310546875, -0.0110931396484375, -0.04437255859375, -0.0355224609375, 0.027252197265625, 0.0297393798828125, 0.01438140869140625, 0.037750244140625, -0.058258056640625, 0.06597900390625, -0.016632080078125, 0.032562255859375, 0.05804443359375, -0.03369140625, 0.0033416748046875, -0.025970458984375, 0.046661376953125, 0.0166168212890625, 0.0034999847412109375, 0.09722900390625, 0.041839599609375, 0.000400543212890625, -0.033966064453125, 0.11273193359375, 0.10235595703125, 0.00649261474609375, -0.0673828125, -0.005817413330078125, -0.1746826171875, -0.04547119140625, -0.056549072265625, -0.01142120361328125, 0.006290435791015625, -0.0257568359375, -0.07537841796875, 0.0132293701171875, 0.05322265625, 0.041107177734375, -0.0088348388671875, 0.03363037109375, 0.0199737548828125, -0.03955078125, -0.035675048828125, -0.07623291015625, -0.004108428955078125, 0.00977325439453125, 0.029632568359375, 0.0160980224609375, -0.0348

In [ ]:
nested_list= embed["data"]
print(embed["data"])
# flat_list1 = [i for s in nested_list for i in s]
print(nested_list[0])

[[-0.0908203125, -0.01268768310546875, -0.0110931396484375, -0.04437255859375, -0.0355224609375, 0.027252197265625, 0.0297393798828125, 0.01438140869140625, 0.037750244140625, -0.058258056640625, 0.06597900390625, -0.016632080078125, 0.032562255859375, 0.05804443359375, -0.03369140625, 0.0033416748046875, -0.025970458984375, 0.046661376953125, 0.0166168212890625, 0.0034999847412109375, 0.09722900390625, 0.041839599609375, 0.000400543212890625, -0.033966064453125, 0.11273193359375, 0.10235595703125, 0.00649261474609375, -0.0673828125, -0.005817413330078125, -0.1746826171875, -0.04547119140625, -0.056549072265625, -0.01142120361328125, 0.006290435791015625, -0.0257568359375, -0.07537841796875, 0.0132293701171875, 0.05322265625, 0.041107177734375, -0.0088348388671875, 0.03363037109375, 0.0199737548828125, -0.03955078125, -0.035675048828125, -0.07623291015625, -0.004108428955078125, 0.00977325439453125, 0.029632568359375, 0.0160980224609375, -0.034881591796875, -0.06683349609375, -0.005664

In [ ]:
embeddings=nested_list[0]
print(nested_list[0])

[-0.0908203125, -0.01268768310546875, -0.0110931396484375, -0.04437255859375, -0.0355224609375, 0.027252197265625, 0.0297393798828125, 0.01438140869140625, 0.037750244140625, -0.058258056640625, 0.06597900390625, -0.016632080078125, 0.032562255859375, 0.05804443359375, -0.03369140625, 0.0033416748046875, -0.025970458984375, 0.046661376953125, 0.0166168212890625, 0.0034999847412109375, 0.09722900390625, 0.041839599609375, 0.000400543212890625, -0.033966064453125, 0.11273193359375, 0.10235595703125, 0.00649261474609375, -0.0673828125, -0.005817413330078125, -0.1746826171875, -0.04547119140625, -0.056549072265625, -0.01142120361328125, 0.006290435791015625, -0.0257568359375, -0.07537841796875, 0.0132293701171875, 0.05322265625, 0.041107177734375, -0.0088348388671875, 0.03363037109375, 0.0199737548828125, -0.03955078125, -0.035675048828125, -0.07623291015625, -0.004108428955078125, 0.00977325439453125, 0.029632568359375, 0.0160980224609375, -0.034881591796875, -0.06683349609375, -0.0056648

In [ ]:
# @title Convert json into array
import json

# Your example data snippet extended to a valid JSON structure
# json_response = '{"data": [[-0.1, 0.2, -0.3], [0.4, -0.5, 0.6], [-0.7, 0.8, -0.9]]}'
json_response = response.json()

# Parse the JSON string into a Python dictionary
data_dict = json.loads(json_response)

# Extract the nested list from the 'data' key
nested_list = data_dict['data']

# Flatten the nested list into a 1D list using a list comprehension
flat_list = [item for sublist in nested_list for item in sublist]

print(flat_list)
# Output: [-0.1, 0.2, -0.3, 0.4, -0.5, 0.6, -0.7, 0.8, -0.9]


TypeError: the JSON object must be str, bytes or bytearray, not dict

In [ ]:
data_2d = response.json()['data']
data_1d = sum(data_2d, [])

In [ ]:
data_1d

[-0.0908203125,
 -0.01268768310546875,
 -0.0110931396484375,
 -0.04437255859375,
 -0.0355224609375,
 0.027252197265625,
 0.0297393798828125,
 0.01438140869140625,
 0.037750244140625,
 -0.058258056640625,
 0.06597900390625,
 -0.016632080078125,
 0.032562255859375,
 0.05804443359375,
 -0.03369140625,
 0.0033416748046875,
 -0.025970458984375,
 0.046661376953125,
 0.0166168212890625,
 0.0034999847412109375,
 0.09722900390625,
 0.041839599609375,
 0.000400543212890625,
 -0.033966064453125,
 0.11273193359375,
 0.10235595703125,
 0.00649261474609375,
 -0.0673828125,
 -0.005817413330078125,
 -0.1746826171875,
 -0.04547119140625,
 -0.056549072265625,
 -0.01142120361328125,
 0.006290435791015625,
 -0.0257568359375,
 -0.07537841796875,
 0.0132293701171875,
 0.05322265625,
 0.041107177734375,
 -0.0088348388671875,
 0.03363037109375,
 0.0199737548828125,
 -0.03955078125,
 -0.035675048828125,
 -0.07623291015625,
 -0.004108428955078125,
 0.00977325439453125,
 0.029632568359375,
 0.0160980224609375,
 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
sims = cosine_similarity(embeddingCheck, data_1d)[0]

ValueError: Expected 2D array, got 1D array instead:
array=[-0.08124869 -0.00964295 -0.02342976 -0.04932073 -0.02408161  0.02553456
  0.03420148  0.00091418  0.0452902  -0.05268493  0.07194553 -0.01487041
  0.03062771  0.04439019 -0.02378064 -0.00205368 -0.00811881  0.03455418
  0.00069404 -0.0072392   0.07818614  0.0409101   0.00306026 -0.02865955
  0.09525371  0.09542746  0.00515423 -0.06141206 -0.01204846 -0.15252656
 -0.0386824  -0.05436026 -0.01303568  0.01286893 -0.02640039 -0.06739054
  0.01444425  0.05156455  0.03713963 -0.00556701  0.0144941   0.0255721
 -0.03123773 -0.02539636 -0.06423905  0.01169506  0.00365119  0.01844604
  0.01836211 -0.01979792 -0.05252086 -0.00179774 -0.01050312  0.05982988
 -0.0129088   0.07150931  0.05784401  0.03204722 -0.00831991  0.06881355
  0.04673234  0.01923073 -0.13168831  0.11012009  0.00074295  0.00628599
 -0.02679921 -0.02452196  0.00693306 -0.03647404 -0.01740911  0.02066136
  0.02585397  0.00490555  0.00671731 -0.02256131 -0.01437996 -0.02

In [ ]:
embeddingCheck=model.encode(query)

In [ ]:
embeddingCheck

array([-0.08124869, -0.00964295, -0.02342976, -0.04932073, -0.02408161,
        0.02553456,  0.03420148,  0.00091418,  0.0452902 , -0.05268493,
        0.07194553, -0.01487041,  0.03062771,  0.04439019, -0.02378064,
       -0.00205368, -0.00811881,  0.03455418,  0.00069404, -0.0072392 ,
        0.07818614,  0.0409101 ,  0.00306026, -0.02865955,  0.09525371,
        0.09542746,  0.00515423, -0.06141206, -0.01204846, -0.15252656,
       -0.0386824 , -0.05436026, -0.01303568,  0.01286893, -0.02640039,
       -0.06739054,  0.01444425,  0.05156455,  0.03713963, -0.00556701,
        0.0144941 ,  0.0255721 , -0.03123773, -0.02539636, -0.06423905,
        0.01169506,  0.00365119,  0.01844604,  0.01836211, -0.01979792,
       -0.05252086, -0.00179774, -0.01050312,  0.05982988, -0.0129088 ,
        0.07150931,  0.05784401,  0.03204722, -0.00831991,  0.06881355,
        0.04673234,  0.01923073, -0.13168831,  0.11012009,  0.00074295,
        0.00628599, -0.02679921, -0.02452196,  0.00693306, -0.03

In [ ]:
list1={}
list1=response.json()

In [ ]:
import requests

url = "URL"

payload = {
    "text": f"""
    {query}
    {results}

    Act like a dsa expert and answer the question. this results given to you.
    """
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print("Status Code:", response.status_code)
print("Response JSON:")
print(response.json())

Status Code: 200
Response JSON:
{'response': '## Linked Lists in DSA\n\nA linked list is a linear data structure consisting of a series of nodes, where each node has a value and a pointer to the next node in the list. The last node in the list has a null pointer, indicating the end of the list.\n\n### Characteristics of Linked Lists in DSA\n\n*   **Insertion**: O(1) - Insertion at the head or tail of the list is a constant time operation, as we maintain head and tail pointers.\n*   **Deletion**: O(n) - Deletion of a node at a specific position in the list requires traversing the list to find the node, resulting in a linear time complexity.\n*   **Searching**: O(n) - Searching for a specific node in the list also requires traversing the list, resulting in a linear time complexity.\n\n### Types of Linked Lists\n\n*   **Singly Linked List**: Each node has a pointer to the next node in the list.\n*   **Doubly Linked List**: Each node has pointers to both the previous and next nodes in the 

In [ ]:
# @title Download Chunks
# Make sure you have `chunks` already loaded (e.g., from extract_chunks_from_pdfs)
# Optional: also use `sources` if you want to track which file the chunk came from

output_file = "/content/DSAdata_chunks.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for i, chunk in enumerate(chunks):
        f.write(f"--- Chunk {i+1} ---\n")
        f.write(chunk.strip())
        f.write("\n\n")  # Add spacing between chunks

print(f"Saved {len(chunks)} chunks to {output_file}")


Saved 611 chunks to /content/DSAdata_chunks.txt


In [ ]:
from google.colab import files

files.download('/content/DSAdata_chunks.txt')
# files.download("bge_data.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title Testing setup
query="Explain Quick Sort in DSA"

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 67.0 MB/s eta 0:00:00


In [ ]:
import faiss
import pickle

In [ ]:
import time

In [ ]:
# @title Full RAG System Testing

#import requirements
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# from dotenv import load_dotenv

# --- Load environment variables ---
# load_dotenv()
# Embedding_URL = os.getenv("Embedding_URL")
# LLM_URL = os.getenv("LLM_URL")

Embedding_URL = "URL"
LLM_URL = "URL"

if not Embedding_URL:
    raise ValueError("Missing Embedding URL in env variables")
if not LLM_URL:
    raise ValueError("Missing LLM URL in env variables")

# Load constant file like faiss and pkl
faiss_file="/content/dsa_index.faiss"
pkl_file="/content/dsa_metadata.pkl"

# Load FAISS index
index = faiss.read_index("/content/dsa_index.faiss")

# Load metadata
with open("/content/dsa_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

chunks = metadata["chunks"]

# Setup Embedding code function
def make_embed(query:str):

    payload={
        "text":f"""{query}"""
    }
    headers={
        "Content-Type": "application/json"
    }

    response=requests.post(Embedding_URL,json=payload,headers=headers)

    res=response.json()

    embed=res["data"]

    embeddings=embed[0]

    return np.array([embeddings]).astype("float32")

# Search for match embeds
def search_embed(query,top_k=5):
    query_vector=make_embed(query)
    distance, indices=index.search(query_vector,top_k)

    results=[]
    # Fix: Iterate over the first (and likely only) result set of indices
    for i in indices[0]:
        results.append({
            "DSA Points":chunks[i]
        })
    return results

def llm_process(query:str,results:str):
    prompt=f"""
You are a Data Structure Algorithm DSA expert.Based on user query and retrived points from DSA book.
##User Query:
{query}
##Retrieved Points:
{results}

## Task understand and answer the query.
"""

#change payload and prompt place holders and variables

    payload={
        "text":prompt
    }

    headers={
        "Content-Type": "application/json"
    }

    response=requests.post(LLM_URL,json=payload,headers=headers)
    res=response.json()

    dec=res["response"]
    decision=dec        #no dec[0] because dec directly load res["response"]

    return decision

def start_process(query):
    try:
        start_time = time.perf_counter()

        top_matchs=search_embed(query)
        retrieved="\n\n".join([item["DSA Points"] for item in top_matchs]) # Extract only the 'DSA Points' values
        decision_llm=llm_process(query,retrieved)

        execution_time = time.perf_counter() - start_time

        return {
            "query": query,
            "embed": top_matchs,
            "retrieved_clauses": retrieved,
            "decision": decision_llm,
            "time": f"{execution_time:.4f}s"
        }

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

x=start_process(query)
print(x)


{'query': 'Explain Quick Sort in DSA', 'embed': [{'DSA Points': '```\n 7) m ← list .Count / 2 8) left ← list( m ) 9) right ← list( list .Count − m ) 10) for i ← 0 to left .Count − 1 11) left [ i ] ← list[ i ] 12) end for 13) for i ← 0 to right .Count − 1 14) right [ i ] ← list[ i ] 15) end for 16) left ← Mergesort( left ) 17) right ← Mergesort( right ) 18) return MergeOrdered( left , right ) 19) end Mergesort\n\n\n<!-- Page 76 -->\n\nCHAPTER 8. SORTING 65\n\n54\n\n2\n\n74\n\n75\n\n4\n\n75\n\n4\n\n54\n\n2\n\n74\n\n4\n\n75\n\n74\n\n54\n\n2\n\n2\n\n5\n\n4\n\nDivide\n\n54\n\n2\n\n75\n\n4\n\n74\n\n54\n\n2\n\n75\n\n74\n\n54\n\n4\n\n2\n\nImpera (Merge)\n\nFigure 8.2: Merge Sort Divide et Impera Approach\n\n8.3 Quick Sort\n\nQuick sort is one of the most popular sorting algorithms based on divide et impera strategy, resulting in an O ( n log n ) complexity. The'}, {'DSA Points': '## Algorithms\n\n62\n\n\n<!-- Page 74 -->\n\nChapter 8\n\n## Sorting\n\nAll the sorting algorithms in this chapter 

In [ ]:
x["query"]
x["time"]

'3.1399s'

In [ ]:
x["embed"]

[{'DSA Points': 'SORTING 64\n4 75 74 2 54 4 75 74 2 54 4 74 75 2 54 4 74 2 75 54 4 74 2 54 75\n0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4\n4 74 2 54 75 4 74 2 54 75 4 2 74 54 75 4 2 54 74 75\n0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4\n4 2 54 74 75 2 4 54 74 75 2 4 54 74 75\n0 1 2 3 4 0 1 2 3 4 0 1 2 3 4\n2 4 54 74 75 2 4 54 74 75\n0 1 2 3 4 0 1 2 3 4\n2 4 54 74 75\n0 1 2 3 4\nFigure 8.1: Bubble Sort Iterations\n1) algorithm Mergesort(list)\n2) Pre: list (cid:54)= ∅\n3) Post: list has been sorted into values of ascending order\n4) if list.'},
 {'DSA Points': '1) algorithm BubbleSort(list)\n2) Pre: list (cid:54)=∅\n3) Post: list has been sorted into values of ascending order\n4) for i←0 to list. Count−1\n5) for j ←0 to list. Count−1\n6) if list[i]<list[j]\n7) Swap(list[i],list[j])'},
 {'DSA Points': '8.1 Bubble Sort\nOne of the most simple forms of sorting is that of comparing each item with\nevery other item in some list, however as the description may imply this form\nof sorting i

In [ ]:
x["retrieved_clauses"]

'SORTING 64\n4 75 74 2 54 4 75 74 2 54 4 74 75 2 54 4 74 2 75 54 4 74 2 54 75\n0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4\n4 74 2 54 75 4 74 2 54 75 4 2 74 54 75 4 2 54 74 75\n0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4\n4 2 54 74 75 2 4 54 74 75 2 4 54 74 75\n0 1 2 3 4 0 1 2 3 4 0 1 2 3 4\n2 4 54 74 75 2 4 54 74 75\n0 1 2 3 4 0 1 2 3 4\n2 4 54 74 75\n0 1 2 3 4\nFigure 8.1: Bubble Sort Iterations\n1) algorithm Mergesort(list)\n2) Pre: list (cid:54)= ∅\n3) Post: list has been sorted into values of ascending order\n4) if list.\n\n1) algorithm BubbleSort(list)\n2) Pre: list (cid:54)=∅\n3) Post: list has been sorted into values of ascending order\n4) for i←0 to list. Count−1\n5) for j ←0 to list. Count−1\n6) if list[i]<list[j]\n7) Swap(list[i],list[j])\n\n8.1 Bubble Sort\nOne of the most simple forms of sorting is that of comparing each item with\nevery other item in some list, however as the description may imply this form\nof sorting is not particularly effecient O(n2). In it’s most s

In [ ]:
print(x["decision"])

**Bubble Sort Explanation in DSA**

Bubble sort is a simple sorting algorithm that works by repeatedly iterating through a list of elements and swapping adjacent elements if they are in the wrong order. This process continues until the list is sorted in ascending order.

**How Bubble Sort Works**

The algorithm consists of two nested loops:

1. The outer loop iterates `n-1` times, where `n` is the number of elements in the list.
2. The inner loop iterates from the first element to the `n-i-1` element.

In each iteration of the inner loop, the algorithm compares adjacent elements and swaps them if they are in the wrong order. This process "bubbles" the largest element to the end of the list.

**Example Walkthrough**

Suppose we have the following list: `[4, 75, 74, 2, 54]`

1. First iteration:
	* Compare 4 and 75, no swap.
	* Compare 75 and 74, swap: `[4, 74, 75, 2, 54]`.
	* Compare 75 and 2, swap: `[4, 74, 2, 75, 54]`.
	* Compare 75 and 54,
